In [1]:
def get_rectangle():
    import random

    width = random.random()
    height = random.random()

    return width, height, int(width > height)

import torch

In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, num_samples):
        super().__init__()
        self.num_samples = num_samples

    def __getitem__(self, index):
        width, height, label = get_rectangle()

        x = torch.tensor([width, height])
        y = torch.tensor(label)
        return x, y

    def __len__(self):
        return self.num_samples

In [4]:
dataset = Dataset(1000)

print(dataset[0],len(dataset))

(tensor([0.7625, 0.4022]), tensor(1)) 1000


In [6]:
loader = torch.utils.data.DataLoader(dataset = dataset, batch_size=8, shuffle=True, drop_last=True)

len(loader),next(iter(loader))

(125,
 [tensor([[0.2682, 0.8684],
          [0.7141, 0.8181],
          [0.5620, 0.8832],
          [0.0116, 0.0438],
          [0.5275, 0.8807],
          [0.2604, 0.1186],
          [0.2862, 0.8477],
          [0.8112, 0.8935]]),
  tensor([0, 0, 0, 0, 0, 1, 0, 0])])

In [9]:
class Model2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 2),
            torch.nn.Softmax(dim=1),
        )

    def forward(self, x):
        y = self.fc(x)
        return y

In [10]:
model2 = Model2()
print(model2(torch.randn(8,2)))

tensor([[0.5303, 0.4697],
        [0.5120, 0.4880],
        [0.4836, 0.5164],
        [0.5303, 0.4697],
        [0.5190, 0.4810],
        [0.5034, 0.4966],
        [0.5200, 0.4800],
        [0.5271, 0.4729]], grad_fn=<SoftmaxBackward0>)


In [17]:
def train():
    # 优化器
    optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4)

    # 损失函数
    loss_fn = torch.nn.CrossEntropyLoss()

    # 训练
    model2.train()

    for epoch in range(100):
        for i, (x, y) in enumerate(loader):
            out = model2(x)
            loss = loss_fn(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 20 == 0:
            acc = (out.argmax(dim=1) == y).float().mean()
            acc2 = (out.argmax(dim=1) == y).sum().item() / len(y)
            print(f"epoch:{epoch}, loss:{loss.item():.4f}, acc:{acc:.4f}, acc2:{acc2:.4f}")

        torch.save(model2, "model1.model")


In [18]:
train()


epoch:0, loss:0.3171, acc:1.0000, acc2:1.0000
epoch:20, loss:0.3550, acc:1.0000, acc2:1.0000
epoch:40, loss:0.3133, acc:1.0000, acc2:1.0000
epoch:60, loss:0.3162, acc:1.0000, acc2:1.0000
epoch:80, loss:0.3133, acc:1.0000, acc2:1.0000


In [13]:
@torch.no_grad()
def test():
    # 从磁盘加载模型
    model = torch.load('model1.model')

    # 模型进入测试模式,关闭dropout等功能
    model.eval()

    # 获取一批数据
    x, y = next(iter(loader))

    # 模型计算结果
    out = model(x).argmax(dim=1)

    print(out, y)
    print(out == y)

In [14]:
test()

tensor([1, 1, 1, 0, 1, 0, 0, 1]) tensor([1, 1, 1, 0, 1, 0, 0, 1])
tensor([True, True, True, True, True, True, True, True])
